In [1]:
import zarr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [34]:
def load_single_data(i):
    data = zarr.open('H:\Desktop\Q1\python\project\TIL-Group15\dataset\CarFollowing/trainHH.zarr', mode='a')
    start, end = data.index_range[i]
    # get timestamps
    timestamps = data.timestamp[start:end]
    # get position, speed, and acceleration
    
    x_lead = data.lead_centroid[start:end]
    v_lead = data.lead_velocity[start:end]
    a_lead = data.lead_acceleration[start:end]
    id=np.ones((1,x_lead.shape[0]))*i
    x_follow = data.follow_centroid[start:end]
    v_follow = data.follow_velocity[start:end]
    a_follow = data.follow_acceleration[start:end]
    array = np.vstack((id, x_lead, v_lead, a_lead, x_follow, v_follow, a_follow, timestamps))
    return array.T

def load_size(i):
    data = zarr.open('H:\Desktop\Q1\python\project\TIL-Group15\dataset\CarFollowing/trainHH.zarr', mode='a')
    #size_lead = 4.85              # this is for AV
    size_lead = data.lead_size[i] # this is for HV
    size_follow = data.follow_size[i]
    id=np.ones(1)*i
    array = np.vstack((id, size_lead, size_follow)) 
    return array.T

def load_multi_size(i):
    multi_data = None
    for x in range(i + 1):
        single_data1 = load_size(x)
        if x == 0:
            multi_data = single_data1
        else:
            multi_data = np.vstack([multi_data, single_data1])
    return multi_data
def load_multi_data(i):
    multi_data = None
    for x in range(i+1):
        single_data = load_single_data(x)
        if x == 0:
            multi_data = single_data
        else:
            multi_data = np.vstack([multi_data, single_data])
    return multi_data
p=int(input('type in the number you want'))
np.savetxt( "multi_data.csv", load_multi_data(p), delimiter="," )
np.savetxt( "multi_size.csv", load_multi_size(p), delimiter="," )

In [87]:
data=np.loadtxt('multi_data.csv',delimiter="," )
data_size=np.loadtxt('multi_size.csv',delimiter="," )
#data_size.columns = ["ID", "size_lead", "size_follow"]
#data.columns = ["ID","x_lead", "v_lead", "a_lead", "x_follow", "v_follow", "a_follow", "timestamps"]


In [109]:
for i in range(p):
    x_data=np.squeeze(data[np.where(data[:,0]==i),:],0)
    x_size=np.squeeze(data_size[np.where(data_size[:,0]==i),:],0)
    
    x_lead=x_data[:,1]
    x_follow=x_data[:,4]
    size_lead = x_size[:,1]
    size_follow = x_size[:,2]
    A_distance=[]
    B_distance=[]
    for y in range(1,x_lead.size):
        A = x_lead[y] - x_follow[y] - 0.5 * (size_lead[0] + size_follow[0])
        B = x_lead[y] - x_follow[y] - 0.5 * (size_follow[0] - size_lead[0])
        A_distance.append(A)
        B_distance.append(B)
    array = np.vstack((A_distance,B_distance))
    np.savetxt( "size.csv", array.T, delimiter="," ) 









